In [ ]:
# https://github.com/h2oai/sql-sidekick/releases
#!pip install --force-reinstall sql_sidekick-0.x.x-py3-none-any.whl

In [ ]:
%load_ext autoreload
%autoreload 2

In [19]:
from sidekick import prompter

In [20]:
prompter.__version__

'0.1.9'

In [21]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [22]:
from sidekick.prompter import db_setup, ask
from sidekick.schema_generator import generate_schema
from sidekick.utils import setup_dir, list_db_dialects

In [41]:
base_path = "./"
cache_path = f"{base_path}/var/lib/tmp"
setup_dir(base_path)

In [42]:
list_db_dialects()

['bigquery',
 'clickhouse',
 'duckdb',
 'hive',
 'mysql',
 'oracle',
 'postgres',
 'presto',
 'redshift',
 'snowflake',
 'spark',
 'spark2',
 'sqlite',
 'starrocks',
 'tableau',
 'trino',
 'tsql',
 'databricks',
 'drill',
 'teradata']

In [43]:
# env variables
import os

os.environ['DATABRICKS_HOST'] = "<>.cloud.databricks.com" # e.g. <>.cloud.databricks.com
os.environ['DATABRICKS_CLUSTER_ID'] = "0112-211956-zzzzz" # e.g. 0232-316956-tsltsyz5
os.environ['DATABRICKS_TOKEN'] = ""
os.environ['OPENAI_API_KEY'] = ""

os.environ['H2O_BASE_MODEL_URL'] = 'http://38.128.233.247'
os.environ['H2O_BASE_MODEL_API_KEY'] = ""
os.environ['RECOMMENDATION_MODEL_REMOTE_URL'] = "https://h2ogpte.genai.h2o.ai"  # e.g. https://<>.h2ogpte.h2o.ai
os.environ['RECOMMENDATION_MODEL_API_KEY'] = ""

# Scenario 1

In [ ]:
# 1. New dataset for QnA
# 2. Generate SQL syntax, but no access to Databricks cluster for runtime validation.

HOST_NAME = "localhost"
USER_NAME = "sqlite"
PASSWORD = "abc"
DB_NAME = "querydb"
PORT = "5432"


# Given .csv file, auto-generate schema
# Download dataset --> https://www.kaggle.com/datasets/uom190346a/sleep-health-and-lifestyle-dataset
# Adjust data_path as needed.
data_path = "./examples/demo/sleep_health_and_lifestyle_dataset.csv"
table_name = "sleep_health_eda_demo"

table_schema, table_info_path = generate_schema(data_path=data_path, output_path=f"{cache_path}/{table_name}_table_info.jsonl")

# Set add_sample=False if no need to add rows to the table
response , err = db_setup(
                db_name=DB_NAME,
                hostname=HOST_NAME,
                user_name=USER_NAME,
                password=PASSWORD,
                port=PORT,
                table_info_path=table_info_path,
                table_samples_path=data_path,
                table_name=table_name,
                local_base_path=base_path,
                add_sample=True
            )

In [45]:
table_info_path

'.//var/lib/tmp/sleep_health_eda_demo_table_info.jsonl'

In [46]:
table_schema

[{'Column Name': 'Person_ID', 'Column Type': 'NUMERIC'},
 {'Column Name': 'Gender',
  'Column Type': 'TEXT',
  'Sample Values': ['Male', 'Female']},
 {'Column Name': 'Age', 'Column Type': 'NUMERIC'},
 {'Column Name': 'Occupation',
  'Column Type': 'TEXT',
  'Sample Values': ['Accountant',
   'Salesperson',
   'Engineer',
   'Software Engineer',
   'Nurse',
   'Teacher',
   'Manager',
   'Lawyer',
   'Doctor',
   'Sales Representative']},
 {'Column Name': 'Sleep_Duration', 'Column Type': 'NUMERIC'},
 {'Column Name': 'Quality_of_Sleep', 'Column Type': 'NUMERIC'},
 {'Column Name': 'Physical_Activity_Level', 'Column Type': 'NUMERIC'},
 {'Column Name': 'Stress_Level', 'Column Type': 'NUMERIC'},
 {'Column Name': 'BMI_Category',
  'Column Type': 'TEXT',
  'Sample Values': ['Overweight', 'Normal', 'Obese', 'Normal Weight']},
 {'Column Name': 'Blood_Pressure', 'Column Type': 'TEXT'},
 {'Column Name': 'Heart_Rate', 'Column Type': 'NUMERIC'},
 {'Column Name': 'Daily_Steps', 'Column Type': 'NUMERI

In [47]:
# One could also decide to pass pre-configured table schema via for table creation
# Schema format:

# [{'Column Name': 'Person_ID', 'Column Type': 'NUMERIC'},
#  {'Column Name': 'Gender',
#   'Column Type': 'TEXT',
#   'Sample Values': ['Male', 'Female']},
#  {'Column Name': 'Age', 'Column Type': 'NUMERIC'},
#  {'Column Name': 'Occupation',
#   'Column Type': 'TEXT',
#   'Sample Values': ['Accountant',
#    'Manager',
#    'Sales Representative',
#    'Salesperson',
#    'Lawyer',
#    'Nurse',
#    'Teacher',
#    'Software Engineer',
#    'Doctor',
#    'Scientist']},
#  {'Column Name': 'Sleep_Duration', 'Column Type': 'NUMERIC'},
#  {'Column Name': 'Quality_of_Sleep', 'Column Type': 'NUMERIC'},
#  {'Column Name': 'Physical_Activity_Level', 'Column Type': 'NUMERIC'},
#  {'Column Name': 'Stress_Level', 'Column Type': 'NUMERIC'},
#  {'Column Name': 'BMI_Category',
#   'Column Type': 'TEXT',
#   'Sample Values': ['Overweight', 'Normal', 'Obese', 'Normal Weight']},
#  {'Column Name': 'Blood_Pressure', 'Column Type': 'TEXT'},
#  {'Column Name': 'Heart_Rate', 'Column Type': 'NUMERIC'},
#  {'Column Name': 'Daily_Steps', 'Column Type': 'NUMERIC'},
#  {'Column Name': 'Sleep_Disorder',
#   'Column Type': 'TEXT',
#   'Sample Values': ['None', 'Sleep Apnea', 'Insomnia']}]




# response, err = db_setup(
#                 db_name=DB_NAME,
#                 hostname=HOST_NAME,
#                 user_name=USER_NAME,
#                 password=PASSWORD,
#                 port=PORT,
#                 table_name=table_name,
#                 table_schema = r,
#                 local_base_path=base_path,
#                 add_sample=False
#             )

In [48]:
def query(question: str, table_name, table_info_path: str, sample_qna_path: str, regenerate=False, regenerate_with_options=False):
    """Asks question and returns SQL."""
    base_path = "."
    # self_correction is enabled by default, set to False if not needed.
    res = ask(
        question=question,
        table_info_path=table_info_path,
        sample_queries_path=sample_qna_path,
        table_name=table_name,
        db_dialect="databricks",
        is_command=False,
        model_name="h2ogpt-sql-sqlcoder-34b-alpha",
        is_regenerate=regenerate,
        is_regen_with_options=regenerate_with_options,
        execute_query=False,
        local_base_path=base_path,
    )
    sql_str = None
    if res:
        _tmp = res[0][1].split("```")[1].replace("sql", '')
        sql_str = ' '.join(_tmp.split())
    return sql_str

In [49]:
res = query("What is the average sleep duration for each gender?", table_name=table_name, 
            table_info_path=table_info_path, sample_qna_path=None, regenerate=False)

2024-01-19 09:35:33.851 | INFO     | sidekick.prompter:ask:493 - Table in use: ['sleep_health_eda_demo']
2024-01-19 09:35:33.852 | INFO     | sidekick.prompter:ask:494 - SQL dialect for generation: databricks
2024-01-19 09:35:33.854 | INFO     | sidekick.prompter:ask:527 - Setting context...
2024-01-19 09:35:33.855 | INFO     | sidekick.prompter:ask:528 - Question: What is the average sleep duration for each gender?
2024-01-19 09:35:33.856 | DEBUG    | sidekick.prompter:ask:546 - Table info path: .//var/lib/tmp/sleep_health_eda_demo_table_info.jsonl
2024-01-19 09:35:33.857 | INFO     | sidekick.utils:is_resource_low:353 - Number of GPUs: 1
2024-01-19 09:35:33.857 | DEBUG    | sidekick.utils:is_resource_low:357 - Information on device: 0
2024-01-19 09:35:33.859 | INFO     | sidekick.utils:is_resource_low:360 - Total Memory: 23GB
2024-01-19 09:35:33.860 | INFO     | sidekick.utils:is_resource_low:361 - Free GPU memory: 22GB
2024-01-19 09:35:33.860 | DEBUG    | sidekick.utils:load_embeddi

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-19 09:35:42.948 | DEBUG    | sidekick.utils:semantic_search:166 - Similarity score for: if patterns like 'current time' or 'now' occurs in question: 0.8459207869447033
2024-01-19 09:35:42.950 | DEBUG    | sidekick.utils:semantic_search:166 - Similarity score for: if patterns like 'total number', or 'list' occurs in question: 0.8319947353454415
2024-01-19 09:35:42.952 | DEBUG    | sidekick.utils:semantic_search:166 - Similarity score for: detailed summary: 0.8346069603076574
2024-01-19 09:35:42.953 | DEBUG    | sidekick.utils:semantic_search:166 - Similarity score for: summary: 0.8394152180082535
2024-01-19 09:35:42.954 | DEBUG    | sidekick.utils:semantic_search:182 - Sorted context: []
2024-01-19 09:35:42.955 | DEBUG    | sidekick.query:generate_sql:469 - Filter Context: []
2024-01-19 09:35:42.956 | INFO     | sidekick.query:generate_sql:477 - Filtering Question/Query pairs ...
2024-01-19 09:35:42.959 | INFO     | sidekick.query:generate_sql:479 - Number of context queries fou

INFO:httpx:HTTP Request: POST http://38.128.233.247:5000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://38.128.233.247:5000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://38.128.233.247:5000/v1/chat/completions "HTTP/1.1 200 OK"


2024-01-19 09:35:45.369 | DEBUG    | sidekick.query:generate_sql:618 - Generated tokens: 
Gender, AVG(Sleep_Duration) AS Average_Sleep_Duration FROM sleep_health_eda_demo GROUP BY Gender ORDER BY Gender DESC;

2024-01-19 09:35:45.375 | INFO     | sidekick.prompter:ask:601 - Input query: What is the average sleep duration for each gender?
2024-01-19 09:35:45.377 | INFO     | sidekick.prompter:ask:602 - Generated response:

SELECT `Gender`, AVG(`Sleep_Duration`) AS `Average_Sleep_Duration` FROM `sleep_health_eda_demo` GROUP BY `Gender` ORDER BY `Gender` DESC LIMIT 100
2024-01-19 09:35:45.384 | INFO     | sidekick.prompter:ask:628 - Alternate responses:

[]


Exiting...


In [50]:
res

'SELECT `Gender`, AVG(`Sleep_Duration`) AS `Average_Sleep_Duration` FROM `sleep_health_eda_demo` GROUP BY `Gender` ORDER BY `Gender` DESC LIMIT 100'

# Scenario 2

In [51]:
# 1. Connect to pre-configured Catalog/Schema on Databricks cluster
# 2. Retrieve information
# 3. Generate SQL syntax, execute against configured DB for the final result

In [52]:
# Retrieve information
from sidekick.db_config import DBConfig

DBConfig.dialect = "databricks"
# Using a demo dataset from Databricks Catalog
config_args = {
    "catalog": "samples",
    "schema": "nyctaxi",
    "cluster_id": os.environ.get("DATABRICKS_CLUSTER_ID")
}
table_name = "trips" # sample table related to NYC Taxi dataset
DBConfig.table_name = table_name
column_info, table_info_path = DBConfig.get_column_info(output_path=f"{cache_path}/{table_name}_table_info.jsonl", **config_args)

INFO:databricks.sql.client:Successfully opened session 89def2e0-902d-4230-91f3-0b2f8404e628
Successfully opened session 89def2e0-902d-4230-91f3-0b2f8404e628
Successfully opened session 89def2e0-902d-4230-91f3-0b2f8404e628


In [53]:
input_q = "Compute average trip distance"
result, _, error = ask(
        question=input_q,
        table_info_path=table_info_path,
        sample_queries_path=None,
        table_name=table_name,
        is_command=False,
        model_name="h2ogpt-sql-sqlcoder-34b-alpha",
        db_dialect="databricks",
        execute_db_dialect="databricks",
        is_regenerate=False,
        is_regen_with_options=False,
        execute_query=True,
        local_base_path=base_path,
        debug_mode=False,
        guardrails=True,
        self_correction=True
    )

2024-01-19 09:36:07.870 | INFO     | sidekick.prompter:ask:493 - Table in use: ['trips']
2024-01-19 09:36:07.871 | INFO     | sidekick.prompter:ask:494 - SQL dialect for generation: databricks
2024-01-19 09:36:07.873 | INFO     | sidekick.prompter:ask:527 - Setting context...
2024-01-19 09:36:07.874 | INFO     | sidekick.prompter:ask:528 - Question: Compute average trip distance
2024-01-19 09:36:07.875 | DEBUG    | sidekick.prompter:ask:546 - Table info path: .//var/lib/tmp/trips_table_info.jsonl
2024-01-19 09:36:07.876 | INFO     | sidekick.utils:is_resource_low:353 - Number of GPUs: 1
2024-01-19 09:36:07.877 | DEBUG    | sidekick.utils:is_resource_low:357 - Information on device: 0
2024-01-19 09:36:07.878 | INFO     | sidekick.utils:is_resource_low:360 - Total Memory: 23GB
2024-01-19 09:36:07.879 | INFO     | sidekick.utils:is_resource_low:361 - Free GPU memory: 22GB
2024-01-19 09:36:07.880 | DEBUG    | sidekick.utils:load_embedding_model:98 - Loading embedding model from: .//models/

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: .//models/sentence_transformers/models--BAAI--bge-base-en/snapshots/17d124a4b773c4c9248ca816b0b0901e3c49a243/
Load pretrained SentenceTransformer: .//models/sentence_transformers/models--BAAI--bge-base-en/snapshots/17d124a4b773c4c9248ca816b0b0901e3c49a243/
Load pretrained SentenceTransformer: .//models/sentence_transformers/models--BAAI--bge-base-en/snapshots/17d124a4b773c4c9248ca816b0b0901e3c49a243/
INFO:databricks.sql.client:Successfully opened session 874a4b41-e623-4e85-88fe-9122124ac500
Successfully opened session 874a4b41-e623-4e85-88fe-9122124ac500
Successfully opened session 874a4b41-e623-4e85-88fe-9122124ac500


2024-01-19 09:36:11.779 | INFO     | sidekick.utils:_check_file_info:467 - Using information info from path .//var/lib/tmp/trips_table_info.jsonl
2024-01-19 09:36:11.781 | INFO     | sidekick.prompter:ask:596 - Computing user request ...
2024-01-19 09:36:13.043 | DEBUG    | sidekick.utils:semantic_search:153 - Input questions: # query: compute average trip distance


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

INFO:databricks.sql.client:Closing session 2371a48a-b7f8-4497-b844-68fa41ec06f2
Closing session 2371a48a-b7f8-4497-b844-68fa41ec06f2
Closing session 2371a48a-b7f8-4497-b844-68fa41ec06f2


2024-01-19 09:36:30.689 | INFO     | sidekick.prompter:ask:666 - Attempting to auto-correct the query...
2024-01-19 09:36:30.690 | DEBUG    | sidekick.prompter:ask:669 - Attempt: 1


Please install the correct version of H2OGPTE with `pip install h2ogpte==1.3.0-dev3`.
You can enable strict version checking by passing strict_version_check=True.
INFO:databricks.sql.client:Successfully opened session 856311f5-20ca-45a9-b58d-b3ea0a427ecc
Successfully opened session 856311f5-20ca-45a9-b58d-b3ea0a427ecc
Successfully opened session 856311f5-20ca-45a9-b58d-b3ea0a427ecc


2024-01-19 09:36:48.508 | DEBUG    | sidekick.db_config:execute_query:268 - Executing query:
 SELECT AVG(`trip_distance`) AS `average_trip_distance` FROM `trips` LIMIT 100


INFO:databricks.sql.client:Closing session 856311f5-20ca-45a9-b58d-b3ea0a427ecc
Closing session 856311f5-20ca-45a9-b58d-b3ea0a427ecc
Closing session 856311f5-20ca-45a9-b58d-b3ea0a427ecc
The query results are:
 [(2.8528291993434256,)]
Exiting...


In [ ]:
?ask

In [54]:
result

['**Generated response for question,**\nCompute average trip distance\n',
 '``` sql\nSELECT AVG(`trip_distance`) AS `average_trip_distance`\nFROM `trips`\nLIMIT 100\n```\n\n',
 '\n',
 '**Result:** \n',
 '[(2.8528291993434256,)]',
 '\n']

In [55]:
# One can make further changes and re-execute the query against the configured Database as required.
DBConfig.execute_query("SELECT SUM(`trip_distance`) AS `average_trip_distance`\nFROM `trips`\nLIMIT 1")

INFO:databricks.sql.client:Successfully opened session 84ea4e01-818d-4f1c-a4a2-7dbb94cd646a
Successfully opened session 84ea4e01-818d-4f1c-a4a2-7dbb94cd646a
Successfully opened session 84ea4e01-818d-4f1c-a4a2-7dbb94cd646a


2024-01-19 09:37:58.015 | DEBUG    | sidekick.db_config:execute_query:268 - Executing query:
 SELECT SUM(`trip_distance`) AS `average_trip_distance`
FROM `trips`
LIMIT 1


INFO:databricks.sql.client:Closing session 84ea4e01-818d-4f1c-a4a2-7dbb94cd646a
Closing session 84ea4e01-818d-4f1c-a4a2-7dbb94cd646a
Closing session 84ea4e01-818d-4f1c-a4a2-7dbb94cd646a


([(62568.24999999994,)],
 'SELECT SUM(`trip_distance`) AS `average_trip_distance`\nFROM `trips`\nLIMIT 1')